In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src.features_engineering import features_engineering

# I. Data Cleaning

## 1. CSI 300 Data processing

In [2]:
csi = pd.read_csv('data/CSI.csv', sep=',')

csi['Date'] = pd.to_datetime(csi['Date'])

csi = csi.set_index('Date')

csi = csi.sort_index(ascending=True)

In [3]:
for col in ['Price', 'Open', 'High', 'Low']:
    csi[col] = csi[col].str.replace(',', '').astype(float)

csi['Volume'] = csi['Vol.'].str.replace('K', 'e3').str.replace('M', 'e6').astype(float)

csi['Close'] = csi['Price']
csi['returns'] = np.log(csi['Close'] / csi['Close'].shift(1))
csi['realized_vol'] = csi['returns'].rolling(20).std() * np.sqrt(252)

In [4]:
csi = csi.drop(columns=['Price', 'Change %', 'Vol.'])

In [5]:
csi = csi[['Open', 'Close', 'High', 'Low', 'Volume', 'returns', 'realized_vol']]

In [6]:
csi

,Open,Close,High,Low,Volume,returns,realized_vol
Date,,,,,,,
2014-10-15,2444.54,2463.87,2465.50,2431.19,135800.0,NaN,NaN
2014-10-16,2448.97,2444.39,2480.30,2441.82,158170.0,-0.007938,NaN
2014-10-17,2443.22,2441.73,2456.11,2410.40,132760.0,-0.001089,NaN
2014-10-20,2448.45,2454.71,2456.62,2441.90,91440.0,0.005302,NaN
2014-10-21,2453.41,2433.39,2459.26,2432.55,96240.0,-0.008723,NaN
...,...,...,...,...,...,...,...
2025-12-25,4629.32,4642.54,4651.15,4621.92,139430.0,0.001828,0.120365
2025-12-26,4639.79,4657.24,4676.13,4635.10,175400.0,0.003161,0.120470
2025-12-29,4658.27,4639.37,4672.04,4629.06,200400.0,-0.003844,0.116328


## 2. PM 2.5 Pollution levels data processing

In [7]:
pol = pd.read_csv('data/beijing-air-quality.csv')
pol['date'] = pd.to_datetime(pol['date'])
pol = pol.rename(columns={'date':'Date'})
pol = pol.set_index('Date')

In [8]:
pol = pol['2014-10-15':'2026-01-01']

In [9]:
pol[' pm25'] = pd.to_numeric(pol[' pm25'], errors='coerce')
pol[' pm10'] = pd.to_numeric(pol[' pm10'], errors='coerce')

In [10]:
pol

,pm25,pm10,o3,no2,so2,co
Date,,,,,,
2014-10-15,108.0,72.0,33,27,5,5
2014-10-16,106.0,118.0,24,54,6,10
2014-10-17,178.0,172.0,51,64,11,20
2014-10-18,251.0,237.0,30,59,5,22
2014-10-19,338.0,143.0,15,38,1,13
...,...,...,...,...,...,...
2025-12-26,112.0,60.0,10,20,2,7
2025-12-27,128.0,87.0,3,32,2,14
2025-12-28,165.0,NaN,,,,


## 3. Control Variables

In [11]:
vix = pd.read_csv('data/VIX.csv', sep=',')

vix = vix.iloc[2:]

vix = vix.rename(columns={'Price': 'Date'})

vix = vix.set_index('Date')

vix = vix.drop(columns=['Volume'])

vix['Close'] = pd.to_numeric(vix['Close'])

vix = vix[['Close']].rename(columns={'Close': 'vix'}).round(4)

In [12]:
vix

,vix
Date,
2014-10-15,25.27
2014-10-16,25.20
2014-10-17,21.99
2014-10-20,18.57
2014-10-21,16.08
...,...
2025-12-24,13.47
2025-12-26,13.60
2025-12-29,14.20


In [13]:
usdcnh = pd.read_csv('data/USDCNH.csv')

usdcnh['Date'] = pd.to_datetime(usdcnh['Date'])

usdcnh = usdcnh.set_index('Date')

usdcnh = usdcnh.sort_index(ascending=True)

usdcnh = usdcnh.iloc[:-1]

usdcnh = usdcnh.rename(columns={'Price': 'Close',
                                'Vol.': 'Volume'})

usdcnh['Close'] = pd.to_numeric(usdcnh['Close'])

usdcnh = usdcnh[['Close']].rename(columns={'Close': 'usdcnh'})

In [14]:
usdcnh

,usdcnh
Date,
2014-10-15,6.1349
2014-10-16,6.1430
2014-10-17,6.1414
2014-10-20,6.1332
2014-10-21,6.1275
...,...
2025-12-25,6.9996
2025-12-26,7.0039
2025-12-29,6.9977


## 4. Data Merging

In [15]:
# Merge sur les trading days du CSI 300
df = csi.join(pol[[' pm25', ' pm10']], how='left')

df = df.rename(columns={' pm25': 'pm25', ' pm10': 'pm10'})

In [16]:
df = df.drop(columns=['Open', 'High', 'Low'])

In [17]:
vix.index = pd.to_datetime(vix.index)
usdcnh.index = pd.to_datetime(usdcnh.index)

df = df.join(vix, how='left')
df = df.join(usdcnh, how='left')

In [18]:
df = df.dropna()

In [19]:
df.to_csv('data/csi300_pollution_df.csv', index='Date')

In [20]:
df

,Close,Volume,returns,realized_vol,pm25,pm10,vix,usdcnh
Date,,,,,,,,
2014-11-12,2594.32,184690.0,0.013860,0.163110,26.0,38.0,13.02,6.1327
2014-11-13,2579.75,219820.0,-0.005632,0.161334,79.0,53.0,13.79,6.1296
2014-11-14,2581.09,165050.0,0.000519,0.160934,113.0,136.0,13.31,6.1301
2014-11-17,2567.10,158480.0,-0.005435,0.163195,73.0,82.0,13.99,6.1262
2014-11-18,2541.42,142940.0,-0.010054,0.164444,161.0,256.0,13.86,6.1178
...,...,...,...,...,...,...,...,...
2025-12-19,4568.18,154880.0,0.003375,0.121971,153.0,8.0,14.91,7.0337
2025-12-22,4611.62,165250.0,0.009464,0.124929,117.0,36.0,14.08,7.0307
2025-12-23,4620.73,169260.0,0.001973,0.121614,81.0,16.0,14.00,7.0190
